In [2]:
import experience_study as exp
import os
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from config import db, app

from models import Hive, ExplanatoryVariable
from datetime import datetime

import data_processing as dclean

In [3]:
explanatory_variables, hives = dclean.collect_data(app)
hives_mod = dclean.rename_ids(hives)
df_normalized = dclean.normalize_data(hives_mod)

df_normalized.head()

,larvae_count,humidity,egg_count,inspection_id,feeding,super_count,temp,stores,activity_surrounding_hive,local_bloom,...,hive_id,date_introduced,replacement_cause,species,origin,date_added,material,user_id,location_long,location_lat
0,Low,74.0,Medium,1,None,3,19.9,Medium,LOW,HIGH,...,1,2024-02-22,n/a,Russian,swarm cells,2025-01-29,Polystyrene,1,78.517182,5.601693
1,Medium,84.3,Low,2,Pollen patty,1,23.3,Low,LOW,LOW,...,1,2024-02-22,n/a,Russian,swarm cells,2025-01-29,Polystyrene,1,78.517182,5.601693
2,Medium,57.9,High,3,Sugar syrup,3,31.1,High,LOW,LOW,...,1,2024-02-22,n/a,Russian,swarm cells,2025-01-29,Polystyrene,1,78.517182,5.601693
3,Low,71.8,Low,4,Pollen patty,3,31.2,Low,HIGH,MEDIUM,...,1,2024-02-22,n/a,Russian,swarm cells,2025-01-29,Polystyrene,1,78.517182,5.601693
4,Medium,72.6,Medium,5,Other,2,21.4,Medium,HIGH,MEDIUM,...,1,2024-02-22,n/a,Russian,swarm cells,2025-01-29,Polystyrene,1,78.517182,5.601693


In [7]:
death_count = df_normalized[['fate', 'date_checked']].loc[df_normalized['fate']=='Dead']
len(death_count)

54